In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2 as cv
import os
from  tqdm import tqdm
from keras.applications import InceptionV3

In [2]:
base_dir = r'C:\Users\91976\Desktop\programming\AI and Ml\projects\sign language detector'

In [3]:
train_dir = fr'{base_dir}\asl_dataset'

In [4]:
zero = fr'{train_dir}\0'
one = fr"{train_dir}\1"
two = fr'{train_dir}\2'
three = fr'{train_dir}\3'
four = fr'{train_dir}\4'
five = fr'{train_dir}\5'
six = fr'{train_dir}\6'
seven = fr'{train_dir}\7'
eight = fr'{train_dir}\8'
nine = fr'{train_dir}\9'

a = fr'{train_dir}\a'
b = fr'{train_dir}\b'
c = fr'{train_dir}\c'
d = fr'{train_dir}\d'
e = fr'{train_dir}\e'
f = fr'{train_dir}\f'
g = fr'{train_dir}\g'
h = fr'{train_dir}\h'
i = fr'{train_dir}\i'
j = fr'{train_dir}\j'
k = fr'{train_dir}\k'
l = fr'{train_dir}\l'
m = fr'{train_dir}\m'
n = fr'{train_dir}\n'
o = fr'{train_dir}\o'
p = fr'{train_dir}\p'
q = fr'{train_dir}\q'
r = fr'{train_dir}\r'
s = fr'{train_dir}\s'
t = fr'{train_dir}\t'
u = fr'{train_dir}\u'
v = fr'{train_dir}\v'
w = fr'{train_dir}\w'
x = fr'{train_dir}\x'
y = fr'{train_dir}\y'
z = fr'{train_dir}\z'


In [5]:
labels = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']


train = []
y_train=[]

img_size = 224
for i in labels:
    folderpath = f"{train_dir}\{i}"
    for j in tqdm(os.listdir(folderpath)):
        img = cv.imread(os.path.join(folderpath,j))
        img = cv.resize(img,(img_size,img_size))
        train.append(img)
        y_train.append(i)


train = np.array(train)
y_train = np.array(y_train)        

100%|██████████| 70/70 [00:00<00:00, 436.92it/s]


In [6]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test = train_test_split(train,y_train, test_size=0.1,random_state=101)


y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)



print(X_test.shape)
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)


(252, 224, 224, 3)
(2263, 224, 224, 3)
(2263, 36)
(252, 36)


In [7]:
pre_model = InceptionV3(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

for layers in pre_model.layers:
    layers.trainable = False

last_layer = pre_model.get_layer('mixed7')
last_output = last_layer.output    

In [8]:
from keras.optimizers import RMSprop,Adam
from keras import Model

x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
output = tf.keras.layers.Dense(len(labels),activation='softmax')(x)

model = Model(pre_model.input,output)

model.compile(optimizer=Adam(learning_rate=0.001), loss = 'categorical_crossentropy',metrics = ['accuracy'])


In [9]:
class callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if(logs.get('val_accuracy')>0.98):
            self.model.stop_training=True

In [10]:
from keras.preprocessing.image import ImageDataGenerator

callback = callbacks()

datagen = ImageDataGenerator(
        rescale=(1./255),
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

datagen_val = ImageDataGenerator(rescale=1./255)

model.fit(
        datagen.flow(X_train,y_train),
        epochs=10,
        validation_data=datagen_val.flow(X_train,y_train),
        callbacks = [callback]
)        


Epoch 1/10
71/71 [==============================] - 127s 2s/step - loss: 1.9511 - accuracy: 0.5294 - val_loss: 1.5650 - val_accuracy: 0.5647
Epoch 2/10
71/71 [==============================] - 116s 2s/step - loss: 0.6350 - accuracy: 0.8953 - val_loss: 0.5890 - val_accuracy: 0.9090
Epoch 3/10
71/71 [==============================] - 105s 1s/step - loss: 0.2919 - accuracy: 0.9421 - val_loss: 0.3135 - val_accuracy: 0.9483
Epoch 4/10
71/71 [==============================] - 107s 2s/step - loss: 0.1693 - accuracy: 0.9713 - val_loss: 0.1340 - val_accuracy: 0.9744
Epoch 5/10
71/71 [==============================] - 128s 2s/step - loss: 0.1239 - accuracy: 0.9739 - val_loss: 0.0944 - val_accuracy: 0.9766
Epoch 6/10
71/71 [==============================] - 112s 2s/step - loss: 0.0747 - accuracy: 0.9867 - val_loss: 0.0479 - val_accuracy: 0.9951


In [16]:
#model.save('sign_detector.h5')

In [17]:
'''
tf_model_file = "sign_detector.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

with open(tf_model_file,'wb') as file:
    file.write(tflite_model)
'''    

INFO:tensorflow:Assets written to: C:\Users\91976\AppData\Local\Temp\tmp6gozme0v\assets


INFO:tensorflow:Assets written to: C:\Users\91976\AppData\Local\Temp\tmp6gozme0v\assets


predicting

In [19]:
t = []

img = cv.imread(r"C:\Users\91976\Desktop\programming\AI and Ml\projects\sign language detector\asl_dataset\0\hand1_0_bot_seg_3_cropped.jpeg")

img = cv.resize(img,(224,224))
t.append(img)

t = np.array(t)

In [20]:
t = t / 255.0
pred = model.predict(t)
print(pred)


1/1 [==============================] - 0s 133ms/step
[[7.7936292e-01 3.4921325e-03 3.5518713e-04 5.0290762e-03 7.2953559e-04
  1.4476723e-03 6.5077760e-04 1.4822347e-03 3.4016831e-04 7.9475454e-04
  7.2298932e-04 1.5183931e-03 1.7350221e-02 5.4265250e-04 1.1246139e-02
  8.7692705e-04 1.6936361e-04 1.2675305e-04 8.6672627e-04 1.4492188e-02
  2.3773341e-03 1.0791142e-03 4.6204332e-02 3.9551803e-03 9.7411074e-02
  2.8553771e-04 1.1285101e-03 7.8838132e-04 2.0455609e-03 3.2474363e-05
  6.0581008e-04 2.6206821e-04 2.8217770e-04 8.1671117e-04 5.7785580e-04
  5.5101194e-04]]
